# Sharing demos with others

Install the Transformers and Datasets libraries to run this notebook.

In [14]:
# Install dependencies
!pip install datasets transformers[sentencepiece]
!pip install --upgrade gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached gradio-3.0.15-py3-none-any.whl (5.1 MB)
  Attempting uninstall: gradio
    Found existing installation: gradio 2.9.4
    Uninstalling gradio-2.9.4:
      Successfully uninstalled gradio-2.9.4


In [1]:
# Imports
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [8]:
# Create chatbot application using multiple interface arguements
# Set metadata
title = "Ask Rick a Question"
description = """
The bot was trained to answer questions based on Rick and Morty dialogues. Ask Rick anything!
<img src="https://huggingface.co/spaces/course-demos/Rick_and_Morty_QA/resolve/main/rick.png" width=200px>
"""
article = "Check out [the original Rick and Morty Bot](https://huggingface.co/spaces/kingabzpro/Rick_and_Morty_Bot) that this demo is based off of."

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("ericzhou/DialoGPT-Medium-Rick_v2")
model = AutoModelForCausalLM.from_pretrained("ericzhou/DialoGPT-Medium-Rick_v2")

# Create function for inference
def predict(input, history=[]):
    # tokenize the new input sentence
    new_user_input_ids = tokenizer.encode(input + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([torch.LongTensor(history), new_user_input_ids], dim=-1)

    # generate a response 
    history = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id).tolist()

    # convert the tokens to text, and then split the responses into the right format
    response = tokenizer.decode(history[0]).split("<|endoftext|>")
    response = [(response[i], response[i+1]) for i in range(0, len(response)-1, 2)]  # convert to tuples of list
    return response, history

# Create and launch interface using data above
gr.Interface(
    fn=predict,
    inputs="textbox",
    outputs="text",
    title=title,
    description=description,
    article=article,
    examples=[["What are you doing?"], ["Where should we time travel to?"]],
).launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://59996.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7fe901582a50>,
 'http://127.0.0.1:7863/',
 'https://59996.gradio.app')

In [ ]:
# How to create sharable link
# gr.Interface(classify_image, "image", "label").launch(share=True)

In [5]:
# Create pictionary application
from pathlib import Path
import torch
import gradio as gr
from torch import nn

# Load classes to predict
LABELS = Path("/content/class_names.txt").read_text().splitlines()

# Create skeleton of model architecture
model = nn.Sequential(
    nn.Conv2d(1, 32, 3, padding="same"),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(32, 64, 3, padding="same"),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(64, 128, 3, padding="same"),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(1152, 256),
    nn.ReLU(),
    nn.Linear(256, len(LABELS)),
)

# Load in model weights and set to eval mode
state_dict = torch.load("/content/pictionary_model.bin", map_location="cpu")
model.load_state_dict(state_dict, strict=False)
model.eval()


# Function for prediction
def predict(im):
    x = torch.tensor(im, dtype=torch.float32).unsqueeze(0).unsqueeze(0) / 255.0
    with torch.no_grad():
        out = model(x)
    probabilities = torch.nn.functional.softmax(out[0], dim=0)
    values, indices = torch.topk(probabilities, 5)
    return {LABELS[i]: v.item() for i, v in zip(indices, values)}

In [7]:
# Create interface object
interface = gr.Interface(
    predict,
    inputs="sketchpad",
    outputs="label",
    theme="huggingface",
    title="Sketch Recognition",
    description="Who wants to play Pictionary? Draw a common object like a shovel or a laptop, and the algorithm will guess in real time!",
    article="<p style='text-align: center'>Sketch Recognition | Demo Model</p>",
    live=True,
)

#Launch interface
interface.launch(share=True)

/usr/local/lib/python3.7/dist-packages/gradio/interface.py:286: UserWarning: Currently, only the 'default' theme is supported.
  warnings.warn("Currently, only the 'default' theme is supported.")


Hint: Set streaming=True for Sketchpad component to use live streaming.
Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://15371.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7fe9036c1bd0>,
 'http://127.0.0.1:7862/',
 'https://15371.gradio.app')